In [1]:
from functools import partial
import pandas as pd
import numpy as np
import uproot
import awkward as ak

from wpca import WPCA

import sys
sys.path.append("..")
from hists.dataframe import *

In [2]:
#input_file = '/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/ClusteringAnalysis/CLUE_clusters_single.root'
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v33/cmssw/data/CLUE_clusters.root'
#input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v20/cmssw/sim_proton_v7/CLUE_clusters.root'
for array_i in uproot.iterate(input_file + ":clusters", step_size="10MB", library="ak"):
    series = array_i
    break

In [3]:
comp = DataframeComputations(series)

In [4]:
comp.clusters3D_PCA

In [5]:
comp.clusters3D_merged_2D.set_index(["event", "clus3D_id"]).loc[105, 2]

,,beamEnergy,clus3D_energy,clus3D_size,clus2D_id,beamEnergy_from_2D_clusters,clus2D_x,clus2D_y,clus2D_z,clus2D_energy,clus2D_layer,clus2D_size,clus2D_rho,clus2D_delta,clus2D_pointType
event,clus3D_id,,,,,,,,,,,,,,
105,2,300.0,3.846443,1,44,300.0,3.917991,-0.283772,39.922501,3.846443,19,48,4.070436,2.118941,1


In [6]:
def doPCA(df:pd.DataFrame):
    if df.clus3D_size.iloc[0] <= 1:
        return np.array([], dtype=np.float32)
    array = df[["clus2D_x", "clus2D_y", "clus2D_z"]].to_numpy()
    weights = df[["clus2D_energy", "clus2D_energy", "clus2D_energy"]].to_numpy()
    pca = WPCA(n_components=3)
    try:
        pca.fit(array, weights=weights)
    except FloatingPointError:
        return np.array([], dtype=np.float32)
    return pca.components_[0]

out_df = comp.clusters3D_merged_2D.groupby(["event", "clus3D_id"]).apply(doPCA)

In [7]:
out_df

event  clus3D_id
0      0                [0.028103285, -0.00849539, 0.9995689]
       1                  [0.7669352, -0.5670277, 0.30048272]
       2                [0.07326906, -0.21396351, -0.9740899]
       3                [-0.06763169, -0.34311634, 0.9368547]
       4               [-0.05694688, -0.12926051, 0.98997414]
                                      ...                    
982    1                [0.03981218, -0.5618744, -0.82626414]
983    0               [-0.00435479, -0.004932344, 0.9999784]
       1                  [0.7480717, 0.58343405, 0.31621757]
984    0            [0.0019933558, -0.005361438, -0.99998367]
985    0                [0.01102886, -0.010263324, 0.9998865]
Length: 1300, dtype: object

In [8]:
df = comp.clusters3D_merged_2D.groupby(["event", "clus3D_id"]).get_group((0, 0))
df

,event,clus3D_id,beamEnergy,clus3D_energy,clus3D_size,clus2D_id,beamEnergy_from_2D_clusters,clus2D_x,clus2D_y,clus2D_z,clus2D_energy,clus2D_layer,clus2D_size,clus2D_rho,clus2D_delta,clus2D_pointType
0,0,0,300.0,144.070526,33,0,300.0,2.471932,-1.914578,13.877500,3.747497,1,25,9.289869,6.871735e-02,0
1,0,0,300.0,144.070526,33,2,300.0,2.520951,-1.829756,14.767500,5.297332,2,33,13.746122,9.702762e-02,0
2,0,0,300.0,144.070526,33,4,300.0,2.364095,-1.885765,16.782499,10.562175,3,44,20.796078,1.457531e-01,0
3,0,0,300.0,144.070526,33,6,300.0,2.472410,-1.983294,17.672501,11.852355,4,45,24.538139,2.307170e-01,0
4,0,0,300.0,144.070526,33,9,300.0,2.613553,-1.800786,19.687500,16.081924,5,60,29.648653,7.141134e-02,0
5,0,0,300.0,144.070526,33,11,300.0,2.660975,-1.747394,20.577499,14.190412,6,63,29.784990,3.402823e+38,1
6,0,0,300.0,144.070526,33,12,300.0,2.740317,-1.884999,22.692499,13.549571,7,59,29.075150,1.521874e-01,0
7,0,0,300.0,144.070526,33,14,300.0,2.848009,-1.863694,23.582500,12.381789,8,62,27.658604,1.097790e-01,0
8,0,0,300.0,144.070526,33,15,300.0,2.592862,-1.739836,25.697500,13.545071,9,66,26.523533,6.853016e-02,0
9,0,0,300.0,144.070526,33,18,300.0,2.824363,-1.973783,26.587500,9.576331,10,58,21.674353,1.125999e-01,0


In [9]:
array = df[["clus2D_x", "clus2D_y", "clus2D_z"]].to_numpy()
weights = df[["clus2D_energy", "clus2D_energy", "clus2D_energy"]].to_numpy()

In [10]:
pca = WPCA(n_components=3)
pca.fit(array, weights=weights)
pca.explained_variance_

array([1.9507721e+01, 1.4560146e-02, 9.2640240e-03], dtype=float32)

In [11]:
pca.components_[0]

array([ 0.02810328, -0.00849539,  0.9995689 ], dtype=float32)